# 动作捕捉并储存

In [ ]:
import cv2
import mediapipe as mp
import time
import numpy as np
import pandas as pd
import os
from pyrealsense2 import pyrealsense2 as rs

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# 配置 RealSense
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# 启动 RealSense
pipeline.start(config)

hand_data = []  # 用于保存每个动作的关键点数据
action_id = 0  # 动作的编号
prev_hand_landmarks = None  # 上一帧的手部关键点
prev_time = None  # 上一帧的时间
video_writer = None  # 用于保存视频
speed_threshold = 0.01  # 速度阈值

# 创建视频和csv文件的存储目录
vid_dir = 'vid'
excel_dir = 'excel'
os.makedirs(vid_dir, exist_ok=True)
os.makedirs(excel_dir, exist_ok=True)

with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  
  while True:
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()
    if not color_frame:
      continue

    image = np.asanyarray(color_frame.get_data())

    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hands.process(image)

    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # 计算手腕的移动速度
        if prev_hand_landmarks is not None and prev_time is not None:
          wrist_speed = np.linalg.norm(np.array([hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y]) - 
                                       np.array([prev_hand_landmarks.landmark[0].x, prev_hand_landmarks.landmark[0].y])) / (time.time() - prev_time)
          if wrist_speed > speed_threshold and video_writer is None:
            # 动作开始，开始记录数据并保存视频
            hand_data = []
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            video_writer = cv2.VideoWriter(os.path.join(vid_dir, f'action_{action_id}.mp4'), fourcc, 30, (640, 480))
          elif wrist_speed < speed_threshold and video_writer is not None:
            # 动作结束，保存数据并关闭视频
            df = pd.DataFrame(hand_data)
            df.to_csv(os.path.join(excel_dir, f'action_{action_id}.csv'), index=False)
            video_writer.release()
            video_writer = None
            action_id += 1

        hand_data.append([landmark.x for landmark in hand_landmarks.landmark] +
                         [landmark.y for landmark in hand_landmarks.landmark] +
                         [landmark.z for landmark in hand_landmarks.landmark])

        prev_hand_landmarks = hand_landmarks
        prev_time = time.time()

    if video_writer is not None:
      video_writer.write(image)

    cv2.imshow('MediaPipe Hands', image)
    
    # 按下 'esc' 键停止录制
    if cv2.waitKey(10) & 0xFF == 27:
        break

cv2.destroyAllWindows()
pipeline.stop()


# 分类信息读取

# 识别模型搭建

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization

# 设定视频帧的形状
frame_num = 30  # 时间长度
frame_height = 64  # 高度
frame_width = 64  # 宽度
channels = 3  # 通道数

# 设定分类的类别数
num_classes = 5  # 假设你有5个动作需要分类

# 定义模型
model = Sequential()

# 第一层3D卷积
model.add(Conv3D(64, (3, 3, 3), activation='relu', padding='same', 
                 input_shape=(frame_num, frame_height, frame_width, channels)))
model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))

# 第二层3D卷积
model.add(Conv3D(128, (3, 3, 3), activation='relu', padding='same'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# 第三层3D卷积
model.add(Conv3D(256, (3, 3, 3), activation='relu', padding='same'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# 第四层3D卷积
model.add(Conv3D(256, (3, 3, 3), activation='relu', padding='same'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# 展平
model.add(Flatten())

# 全连接层
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))

# 输出层
model.add(Dense(num_classes, activation='softmax'))

# 编译模型
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

# 打印模型概况
model.summary()
